# Analysis-of-2024-French-Legislatives-Elections

İmport Modules

In [5]:
import pandas as pd
import sqlite3
import google.generativeai as genai
import csv


# Gathering Data

In [ ]:
csv_file = 'resultats-definitifs-par-regions.csv'
df = pd.read_csv(csv_file, sep=';', decimal=',')

# create a SQLite database and connect to it
db_file = 'elections.db'
conn = sqlite3.connect(db_file)

# write the DataFrame to a SQLite table
df.to_sql('elections', conn, if_exists='replace', index=False)
conn.close()
print(f"Data from {csv_file} has been written to {db_file} in the 'elections' table.")


Data from resultats-definitifs-par-regions.csv has been written to elections.db in the 'elections' table.


# Using the LLM to Write and Run SQL